In [31]:
import pandas as pd
import re

In [32]:
pd.set_option('display.max_colwidth', None)

In [33]:
# read data, assign column names, skip header
data = pd.read_csv("depression_dataset_reddit_cleaned.csv", names=["text", "is_depression"], header=0)

In [37]:
# look at some samples
print(data[data["is_depression"] == 1].sample(n=3)["text"])
print(data[data["is_depression"] == 0].sample(n=3)["text"])

749                                                                                                                                                                                                                                                                                                                                                                                                                                                  i think the worst part of dealing with all of this is the absolute constant feeling of loneliness having no connection to anyone many act like they want to help but a soon a they see how deep and dark it is they back off my phone stay pretty silent unless someone need something
3560                                                                                                                                                                                                                                                                                        

In [41]:
def preprocess(text):
    # remove extra blanks
    re.sub(r'\t{2,}', ' ', text)
    # TODO other stuff
    
data["text"].apply(preprocess)

0       None
1       None
2       None
3       None
4       None
        ... 
7726    None
7727    None
7728    None
7729    None
7730    None
Name: text, Length: 7731, dtype: object